In [26]:
import urllib
import urllib.request
import json

import time

import pandas as pd
import numpy as np

In [2]:
binance_base = "https://<>.binance.com"
binance_subdomains = ["api", "api1", "api2", "api3"]

binance_url = binance_base.replace('<>', binance_subdomains[0])

binance_endpoints = {
    'ping': ('GET', '/api/v3/ping'),
    'server_time': ('GET', '/api/v3/time'),
    'exchange_info': ('GET', '/api/v3/exchangeInfo'),
    'order_book': ('GET', '/api/v3/depth', {'symbol': True, 'limit': False}),
    'recent_trades': ('GET', '/api/v3/trades', {'symbol': True, 'limit': False}),
    'average_price': ('GET', '/api/v3/avgPrice', {'symbol': True}),
    'price': ('GET', '/api/v3/ticker/price', {'symbol': False}),
    'best_book_price': ('GET', '/api/v3/ticker/bookTicker', {'symbol': False})
}

In [4]:
def get_symbols():
    binance_uri = binance_url + binance_endpoints['exchange_info'][1]

    web_url = urllib.request.urlopen(binance_uri)
    data = web_url.read()
    encoding = web_url.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))

    df_symbols = pd.DataFrame(JSON_object['symbols'])
    df_symbols = df_symbols[df_symbols['status']=='TRADING']
    
    assets = sorted(list(set(df_symbols['baseAsset'].unique()).union(set(df_symbols['quoteAsset'].unique()))))
    symbols = sorted(list(df_symbols['symbol'].unique()))
    return symbols, assets

symbols, _ = get_symbols() 

In [44]:
def get_symbol_prices(symbol='BTCEUR'):
    binance_uri = binance_url + binance_endpoints['order_book'][1] + f'?symbol={symbol}&limit=5'

    web_url = urllib.request.urlopen(binance_uri)
    data = web_url.read()
    encoding = web_url.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))

    prices = [float(price) for price, quantity in JSON_object['bids']]
    weights = [float(quantity) for price, quantity in JSON_object['bids']]
    buy = np.average(prices, weights=weights)

    prices = [float(price) for price, quantity in JSON_object['asks']]
    weights = [float(quantity) for price, quantity in JSON_object['asks']]
    sell = np.average(prices, weights=weights)

    return buy, sell

In [45]:
time_i = time.time()*1000
print(get_symbol_prices())
time_e = time.time()*1000
time_e-time_i

(48898.17573288813, 48908.44375596953)


358.43408203125